In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
model = tf.keras.models.load_model('model.h5')
classes = ['IN-USE-LEG-EXTENSION','IN-USE-PEC-FLY','NOT-IN-USE-LEG-EXTENSION','NOT-IN-USE-PEC-FLY']
classes

In [ ]:
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (128, 128))
    frame = img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)
    frame = frame / 255.0
    return frame


In [ ]:
def predict_frame(frame, model, video_num):
    prediction = model.predict(frame)
    predicted_class = classes[np.argmax(prediction)]
    
    if video_num == 'Video 1':
        if predicted_class == 'NOT-IN-USE-LEG-EXTENSION':
            return 'not in use'
        else:
            return 'in use'
    elif video_num == 'Video 2':
        if predicted_class == 'NOT-IN-USE-PEC-FLY':
            return 'not in use'
        else:
            return 'in use'
    else:
        return predicted_class

In [ ]:
vid1 = cv2.VideoCapture('D:/MY_FILES/futura/ml/project/pj-DL/video/2024-07-06 21.53.16.mov')
vid2 = cv2.VideoCapture('D:/MY_FILES/futura/ml/project/pj-DL/video/2024-07-06 22.11.52.mov')

if not vid1.isOpened() or not vid2.isOpened():
    print("Error: Could not open one of the videos.")
    exit()

while vid1.isOpened() and vid2.isOpened():
    ret1, frame1 = vid1.read()
    ret2, frame2 = vid2.read()

    if not ret1 or not ret2:
        break

    predicted_vid1 = predict_frame(preprocess_frame(frame1), model, 'Video 1')
    predicted_vid2 = predict_frame(preprocess_frame(frame2), model, 'Video 2')

    frame1 = cv2.resize(frame1, (640, 480))
    frame2 = cv2.resize(frame2, (640, 480))
 
    log_table = np.zeros((100, 640, 3), dtype=np.uint8)
    cv2.putText(log_table, f'LEG-EXTENSION: {predicted_vid1}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(log_table, f'PEC-FLY: {predicted_vid2}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('LEG-EXTENSION', frame1)
    cv2.imshow('PEC-FLY', frame2)
    cv2.imshow('Predictions', log_table)

    if cv2.waitKey(1) == ord('q'):
        break

vid1.release()
vid2.release()
cv2.destroyAllWindows()
